# CRAM to FASTQ processing

In [1]:
%load_ext dotenv

In [2]:
%dotenv

In [3]:
import os
import pandas as pd
import re

## Create sample objects

In [5]:
# Read CRAM manifest
df_cram = pd.read_csv(f"../{os.environ['CRAM_MANIFEST']}", header=None, names=['cram_bucket', 'cram'])
# Extract sample name
df_cram['sample'] = df_cram['cram'].apply(lambda x: re.search(r'([^/]+)\.bqsr', x).group(1))

df_cram.columns

Index(['cram_bucket', 'cram', 'sample'], dtype='object')

In [ ]:
# For each sample, create an ICA sample object
for sample in df_cram['sample']:
    # Create sample object
    os.system(f"icav2 -k $ICA_API_KEY --project-id $ICA_PROJECT_ID projectsamples create {sample}")

In [10]:
# List ICA sample objects
os.system(f"icav2 -k $ICA_API_KEY --project-id $ICA_PROJECT_ID projectsamples list > projectsamples-list.txt")


0

In [11]:
# Read projectsamples-list
df_sample = pd.read_csv('projectsamples-list.txt', sep='\t', header=0, index_col=False, engine='python', skipfooter=1,
                 names=['ID', 'NAME', 'STATUS', 'DESCRIPTION', 'USER TAGS', 'TECHNICAL TAGS']
                )
print(f'n={len(df_sample):,}')
print(df_sample.columns)

n=1,543
Index(['ID', 'NAME', 'STATUS', 'DESCRIPTION', 'USER TAGS', 'TECHNICAL TAGS'], dtype='object')


In [ ]:
# For each sample object that are not completed
for i, row in df_sample.loc[df_sample['STATUS'].str.strip() == 'PARTIAL'].iterrows():
    # Complete the sample
    os.system(f"icav2 -k $ICA_API_KEY --project-id $ICA_PROJECT_ID projectsamples complete {row['ID']}")
    # Tag the sample
    os.system(f"icav2 -k $ICA_API_KEY --project-id $ICA_PROJECT_ID projectsamples update {row['ID']} --add-user-tag {row['NAME']}")
